In [2]:
__author__ = "Elena Caceres"
__email__ = "ecaceres@keiserlab.org"
"""Purpose: generate data for use with training neural networks. DM and PCBA scrubbed"""

'Purpose: generate data for use with training neural networks. DM and PCBA scrubbed'

In [3]:
import gzip
import argparse
import h5py
import cPickle as pkl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import pandas as pd
from collections import defaultdict
from scipy.stats import mode
from common.data_converter import convert_to_pki
np.random.seed(42)

def get_env_var(handle):
    tmp = os.getenv(handle)
    if not tmp:
        raise LookupError("Environment variable: {} not set.".format(handle))
    return tmp.strip("'")

In [4]:
def get_target_mol_pairs(chembl):
    # for ChEMBL
    unique_pairs = set()
    with gzip.open(chembl, "rb") as f:
        a = f.next()
        vals = a.split("\t")
        print("{}: {}".format(vals[2], vals[3]))
        for line in f:
            # tid:mid
            unique_pairs.update({(line.split("\t")[2], line.split("\t")[3])})

    return unique_pairs

In [5]:
base = get_env_var("DATA_SAVE_BASE")
expt_base = "{}/20180525_DM_scrubbing/train_data".format(base)

In [14]:
# load inchi:mid map
print("loading inchi to mid")
with gzip.open(inchi_map, "rb") as f:
    # store molecules in case we want to go back and look them up.
    ikey_map = pkl.load(f)
ikey_map = ikey_map.set_index("index").to_dict()[0]

# load fingerprints:ikey file
fp = ""
fp_lookup_dict={ikey_map[i] : np.zeros((fp_len,), dtype=bool) for i in ikey_map.keys()}
with gzip.open(fp_file, "r") as f:
    for line in f:
        fp, ikey = line.rstrip().split("\t")
        fp_lookup_dict.update({ikey: np.array(map(int, fp), dtype=bool)})
assert(fp_len == len(np.array(map(int, fp), dtype=bool)))

print("Fingerprint length: {}".format(fp_len))

target_mol_pairs = get_target_mol_pairs(chembl_data)

unique_pairs = set()
for tid, mid in target_mol_pairs:
    try:
        unique_pairs.update({(tid, ikey_map[mid])})
    except KeyError:
        print("Skipping MID {} for target {}. Not in INCHI lookup".format(mid, tid))

print("Length of total tid, mid pairs: {}".format(len(unique_pairs)))

# number of unique molecules/target on on a target basis
targets_counts = defaultdict(int)
for pair in unique_pairs:
    targets_counts[pair[0]] += 1

# Get a list of (target, molecule) pairs where the number of unique molecules surpasses 9
# Re-open our original dataset and get a list of tuples of all (values, relation, doc_id, year) for our data
all_data = {}
unique_ikey_smiles = {}
values = []
# get chembl data
with gzip.open(chembl_data, "rb") as f:
    f.next()
    for line in f:
        doc_id, year, tid, mid, pref_name, act, rel, smi = line.split("\t")
        try:
            ikey = ikey_map[mid]
        except KeyError:
            continue
        if (tid, ikey) in unique_pairs:
            if rel == '>':
                act = float(act)
                orig_act = act
                # convert to log space
                act = -np.log10(act) + 9
                # add 2-3 logs
                act -= np.random.uniform(2.0, 3.0)
                if act <= 0:
                    # we're in positive space here, people! can't have a negative, that would be madness.
                    if -act > orig_act:
                        act = orig_act
                    else:
                        act = -act
                # and then convert it back to nM space
                act = 10 ** (-(act - 9))
            if (tid, ikey) in all_data:
                try:
                    all_data[(tid, ikey)].append((int(doc_id), int(year), float(act), rel))
                    values.append(float(act))
                except:
                    all_data[(tid, ikey)].append((int(doc_id), None, float(act), rel))
                    values.append(float(act))
            else:
                try:
                    all_data[(tid, ikey)] = [(int(doc_id), int(year), float(act), rel)]
                    values.append(float(act))
                except:
                    all_data[(tid, ikey)] = [(int(doc_id), None, float(act), rel)]
                    values.append(float(act))
            if ikey not in unique_ikey_smiles:
                unique_ikey_smiles.update({ikey: smi})

# iterate through the values in each pair
# Used median to reduce the importance of outliers. In the case of an even number, it just takes the mean anyways.
consensus = {key: None for key in all_data.keys()}

for key, value in all_data.iteritems():
    # get the median value of binding affinity
    act = np.median(np.asarray([i[2] for i in value]))
    # get the mode of the relation value
    rel = mode(np.asarray([i[3] for i in value]))
    # get the min year
    try:
        year = np.min(np.asarray([i[1] for i in value if i is not None]))
    except:
        year = 0
    consensus[key] = (act, rel[0][0], year)

# only accept targets with at least 10 positive values
# indexer
targets_counts_above_10 = {k: v for (k, v) in targets_counts.items() if v >= 10}
target_pos_count = {k: 0 for k in targets_counts_above_10}

# key[0] = target ; key[1] = molecule
# value [0] = act ; value[1] = rel ; value[2] = year
for key, value in consensus.items():
    if convert_to_pki(value[0]) > cutoff:
        try: 
            target_pos_count[key[0]] += 1
        except KeyError:
            continue

final_target_set = {k for k, v in target_pos_count.items() if v >= 10}

# create lookup table for what positon in the array corresponds to which target
# ran once, for generation now, use saved target index for consistency
target_index_outfile = "{}/{}_target_index.pkl".format(out_dir, output_base_name)
target_index = {}
count = 0
for target in final_target_set:
    target_index.update({target: count})
    count += 1
with open(target_index_outfile, 'wb') as tout:
    pkl.dump(target_index, tout)

bad_molecules = {k for k, v in fp_lookup_dict.items() if v.size == 0}
consensus_minus_bad = {k: v for k, v in consensus.items() if k[1] not in bad_molecules and k[0] in target_index}
print("Bad training cases removed: %d" % (len(consensus) - len(consensus_minus_bad)))

tmp_fp, tmp_act, tmp_pos, tmp_year, multi_task_lookup = write_onemol_onetarget_format(consensus_minus_bad, fp_lookup_dict, target_index, "{}/{}".format(out_dir, output_base_name), fp_len, cutoff=cutoff)


Length of total tid, mid pairs: 709614
Bad training cases removed: 22018
current fcn: one mol one target
Training Cases: 687596
Fingerprint Length: 4096
Number of Targets: 2326
current fcn: one mol many target
Training Cases: 1
Fingerprint Length: 4096
Number of Targets: 2326


In [13]:
!ls $expt_base

chembl20_MWmax800_scrubDM_minpos10_cutoff5_1midtomanytid.hdf5
chembl20_MWmax800_scrubDM_minpos10_cutoff5_onetomany_ikey_lookup_file.pkl
chembl20_MWmax800_scrubDM_minpos10_cutoff5_target_index.pkl
make_hdf5.log


In [7]:
out_dir = "{}/time_split".format(expt_base)
bv_type = "4096"
CUTOFF_YEAR = 2012
fp_len=int(bv_type)

In [8]:
# output files
in_h5py = "{}/chembl20_MWmax800_scrubDM_minpos10_cutoff5_1midtomanytid.hdf5".format(expt_base)
in_target_index = "{}/chembl20_MWmax800_scrubDM_minpos10_cutoff5_target_index.pkl".format(expt_base)


val_hdf5= "{}/val_chembl20_MWmax800_scrubDM_minpos10_cutoff5_1midtomanytid_fplen{}_TS{}.hdf5".format(out_dir, bv_type, CUTOFF_YEAR)
train_hdf5 = "{}/train_chembl20_MWmax800_scrubDM_minpos10_cutoff5_1midtomanytid_fplen{}_TS{}.hdf5".format(out_dir, bv_type, CUTOFF_YEAR)
in_target_index = "{}/chembl20_MWmax800_scrubDM_minpos10_cutoff5_fplen{}_TS{}_target_index.pkl".format(expt_base, bv_type, CUTOFF_YEAR)


In [11]:
# read data to arrays
with h5py.File(in_h5py, 'r') as h:
    act_arr = h["activity"][:].copy()
    fp_arr = h["fp_array"][:].copy()
    pos_arr =  h["position"][:].copy()
    year_arr = h["year"][:].copy()
    print np.shape(act_arr)
    print np.shape(fp_arr)
    print np.shape(pos_arr)
    print np.shape(year_arr)

(1, 1)
(1, 4096)
(1, 1)
(1, 1)


In [10]:
act_arr

array([[ 550.]], dtype=float32)

In [51]:
str(int('CHEMBL1075145'))

ValueError: invalid literal for int() with base 10: 'CHEMBL1075145'

In [47]:
"CHEMBL181880" in new_test

True

In [46]:
new_test = {i[1] for i in unique_pairs}

In [50]:
'CHEMBL181880' in set(ikey_map.keys())

False

In [39]:
with gzip.open(mid2inchi, "rb") as f:
    # store molecules in case we want to go back and look them up.
    ikey_map = pkl.load(f)
ikey_map = ikey_map.set_index("index").to_dict()[0]

In [38]:
chembl_df = pd.read_csv(raw_chembl_name, sep="\t", compression="gzip", index_col=0)
tmp_df = chembl_df.groupby(["ChEMBL_Target_ID", "ChEMBL_Molecule_ID"]).size().reset_index().rename(columns={0:'count'})
unique_pairs = { (i.ChEMBL_Target_ID, i.ChEMBL_Molecule_ID) for i in tmp_df.itertuples()}

In [36]:
chembl_df = pd.read_csv(raw_chembl_name, sep="\t", compression="gzip")

In [7]:
unique_pairs = set()
with gzip.open(raw_chembl_name, "rb") as f:
    a = f.next()
    vals = a.split("\t")
    print("{}: {}".format(vals[2], vals[3]))
    for line in f:
        # tid:mid
        unique_pairs.update({(line.split("\t")[2], line.split("\t")[3])})

year: ChEMBL_Target_ID


In [7]:
print("loading inchi to mid map")
with gzip.open(mid2inchi, "rb") as f:
    # store molecules in case we want to go back and look them up.
    ikey_map = pkl.load(f)
ikey_map = ikey_map.set_index("index").to_dict()[0]

loading inchi to mid map


In [5]:
base = get_env_var("DATA_SAVE_BASE")
expt_base = "{}/20180525_DM_scrubbing".format(base)
raw_chembl_name = "{}/raw_data/full_chembl20_cutoff800_dm_scrubbed.csv.gz".format(expt_base)
smiles = "{}/raw_data/all_chembl_smiles_mid_mwcutoff800.smi".format(expt_base)
inchi_dict="{}/raw_data/chembl20_MWmax800_smiles2inchi2mid.csv.gz".format(expt_base)
inchi2smiles = "{}/raw_data/inchi2smiles.csv.gz".format(expt_base)
mid2inchi = "{}/raw_data/mid2inchi.csv.gz".format(expt_base)

out_base = "{}/train_data".format(expt_base)
output_base_name = "chembl20_MWmax800_scrubDM_minpos10_cutoff5"
fp_file = "{}/raw_data/chembl20_MWmax800_fps.fp.gz".format(expt_base)